In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
#总计20640个样本，每个样本8个属性表示，以及房价作为target，所有属性值均为number
#目标变量：平均房屋价值
#输入变量（特征）：平均收入、住房平均年龄、平均房间、平均卧室、人口、平均占用、纬度和经度

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [3]:
 X_train.shape

(11610, 8)

In [4]:
y_train.shape

(11610, 1)

In [5]:
X_train.shape[1:]

(8,)

In [73]:
input_shape = X_train.shape[1:]

### 自定义损失函数

tf.where(input_tensor, a,b)：对于张量a，如果input_tensor对应位置的元素为True，则张量a中的该位置处元素保留，反之由张量b中相应位置的元素来代替。

In [6]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1 
    #判断，当误差的绝对值<1时loss为误差平方，>1时为线性的
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [7]:
input_shape = X_train.shape[1:]

#输入每个房子的8个特征，预测其价格
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [8]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])
#计算平均绝对误差MAE，使用自定义的huber损失

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.5417 - mae: 0.8956 - val_loss: 0.2505 - val_mae: 0.5390
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2102 - mae: 0.5048 - val_loss: 0.2095 - val_mae: 0.4958


### 保存和加载包含自定义组件的模型

In [9]:
model.save("my_model_with_a_custom_loss.h5")

In [10]:
#keras会保存模型名称，每次加载时需提供一个字典将函数名称映射到实际函数
model=keras.models.load_model("my_model_with_a_custom_loss.h5",
                              custom_objects={"huber_fn":huber_fn})

In [11]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2005 - mae: 0.4916 - val_loss: 0.1932 - val_mae: 0.4764
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.1960 - mae: 0.4855 - val_loss: 0.1862 - val_mae: 0.4657


#### 自定义误差阈值（1）

In [12]:
#创建一个函数重新创建已配置的损失函数，阈值默认值为1.0
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [13]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])
#指定阈值为2.0

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2174 - mae: 0.4853 - val_loss: 0.2231 - val_mae: 0.4748
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2118 - mae: 0.4785 - val_loss: 0.2115 - val_mae: 0.4622


In [14]:
#保存模型时不会保存阈值
model.save("my_model_with_a_custom_loss_threshold_2.h5")

加载自定义损失的模型时必须指定其自定义loss的阈值

In [15]:
#加载时必须指定阈值，字典名称为为Keras命名的函数名称而非创建函数时的名称
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [16]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 0.2098 - mae: 0.4752 - val_loss: 0.2417 - val_mae: 0.4734
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2071 - mae: 0.4705 - val_loss: 0.1993 - val_mae: 0.4526


#### 自定义误差阈值（2）

**kwargs将一个可变关键字参数的字典dict传给函数实参，参数列表长度可以为0或为其他值。

In [17]:
class HuberLoss(keras.losses.Loss):
    
    def __init__(self, threshold=1.0, **kwargs):
        """
         接受**kwargs并将它们传递给父类构造函数，
         其父类构造函数处理标准超参数：损失的name 和
         用于聚合单个实例损失的reduction算法（默认为实例损失的总和）
        """
        self.threshold = threshold
        super().__init__(**kwargs)#继承keras.losses.Loss类的初始化参数
        
    def call(self, y_true, y_pred):
        """获取标签和预测，计算所有实例loss，然后将其返回"""
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        """使用keras.losses.Loss类的get_config()方法
           保存新的超参数（自定义的loss的阈值）
           返回一个字典，将每个超参数名称映射到其值"""
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [18]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])
#将阈值设置为2，使用HuberLoss类创建实例loss

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.8638 - mae: 1.0278 - val_loss: 0.2943 - val_mae: 0.5372
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2499 - mae: 0.5163 - val_loss: 0.2548 - val_mae: 0.5073


In [19]:
#保存模型时keras调用get_config()将配置以JSON格式保存到HDF5文件中
#阈值会同时一起保存，加载时将类名映射到类本身即可
model.save("my_model_with_a_custom_loss_class.h5")

model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

In [20]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 0.2312 - mae: 0.4992 - val_loss: 0.2187 - val_mae: 0.4814
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.2249 - mae: 0.4923 - val_loss: 0.2558 - val_mae: 0.4984


In [21]:
model.loss.threshold

2.0

### 自定义激活函数、初始化、正则化、约束

##### 自定义激活函数

In [22]:
#等同于keras.activations.softplus(z) 或 tf.nn.softplus(z)
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

#### 自定义Glorot初始化

In [23]:
#等同于keras.initializers.glorot_normal()
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)#正态初始化

#### 自定义l1正则化

tf.reduce_sum ():按一定方式计算张量中元素之和

In [24]:
#等同于keras.regularizers.l1(0.01)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

#### 确保权重均为正的自定义约束

tf.nn.relu(features, name = None):计算激活函数 relu即 max(features, 0)

tf.zeros_like（）：给定一个张量(tensor),返回所有元素都设置为零的tensor

In [25]:
#等同于keras.contraints.nonneg():权重非负的约束
#或tf.nn.relu(weights)
def my_positive_weights(weights): 
    """当权重＜0时将其设为0，当权重>=0时不变"""
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [26]:
layer = keras.layers.Dense(1, 
                           activation=my_softplus,
                           #激活函数应用于该层的输出，将结果传递给下一层
                           
                           kernel_initializer=my_glorot_initializer,
                           #初始化层的权重
                           
                           kernel_regularizer=my_l1_regularizer,
                           #训练过程中将权重传递给正则化函数计算正则化损失，
                           #将其添加到主要损失中以得到最终loss
                           
                           kernel_constraint=my_positive_weights
                           #在每个训练步骤后调用，将该层权重替换为约束权重
                          )


In [44]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape))
model.add(layer)
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=2,validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.6712 - mae: 0.8907 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.6286 - mae: 0.5310 - val_loss: inf - val_mae: inf


In [28]:
model.save("my_model_with_many_custom_parts.h5")

#加载模型时指定自定义参数的字典
model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "my_l1_regularizer": my_l1_regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

tf.abs():计算张量的绝对值

In [29]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    """
    用于l1正则化的普通类，
    保存了超参数factor(控制L1权重衰减的程度)
    """
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, weights):
        """将权重与正则化参数相乘，并求和"""
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {"factor": self.factor}

In [30]:
a= tf.constant([[1, 2, 3], [4, 5, 6]])
print(tf.reduce_sum(a))#求得所有元素和21

tf.Tensor(21, shape=(), dtype=int32)


In [31]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=MyL1Regularizer(0.01),
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

In [32]:
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 1s 1ms/step - loss: 1.9280 - mae: 0.9557 - val_loss: inf - val_mae: inf
Epoch 2/2
363/363 [==============================] - 0s 1ms/step - loss: 0.6748 - mae: 0.5337 - val_loss: inf - val_mae: inf


In [33]:
model.save("my_model_with_many_custom_parts.h5")

model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "MyL1Regularizer": MyL1Regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })